In [ ]:
! gdown 1yEjicXasMGFaALQobQWgsSxQgLEtxruC
! unzip /content/ThaiDepression.zip

Downloading...
From: https://drive.google.com/uc?id=1yEjicXasMGFaALQobQWgsSxQgLEtxruC
To: /content/ThaiDepression.zip
100% 41.1k/41.1k [00:00<00:00, 58.9MB/s]
Archive:  /content/ThaiDepression.zip
  inflating: ThaiDepression/AHB_01.csv  
  inflating: ThaiDepression/AHB_02.csv  
  inflating: ThaiDepression/AHB_03.csv  
  inflating: ThaiDepression/AHB_04.csv  
  inflating: ThaiDepression/AHB_05.csv  
  inflating: ThaiDepression/AHB_06.csv  
  inflating: ThaiDepression/AHB_07.csv  
  inflating: ThaiDepression/AHB_08.csv  
  inflating: ThaiDepression/AHB_09.csv  
  inflating: ThaiDepression/AHB_10.csv  
  inflating: ThaiDepression/AHB_11.csv  
  inflating: ThaiDepression/AHB_12.csv  
  inflating: ThaiDepression/AHB_13.csv  
  inflating: ThaiDepression/All_AHB.csv  


In [ ]:

import pandas as pd
import numpy as np
import os
from numpy import array
from math import sqrt
from numpy import mean
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from keras.preprocessing import sequence
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from pandas import read_csv
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, SimpleRNN
import json
import psutil 
from datetime import datetime, timezone, timedelta

In [ ]:
file_list = os.listdir('ThaiDepression')
file_list.sort()
df_list = []
for i in range(14):
    df_list.append(pd.read_csv('ThaiDepression/' + file_list[i], index_col=0))

In [ ]:
df_list

[         Chiang_Rai    Nan  Phayao  Phrae  Chiang_Mai  Mae_Hong_Son  Lampang  \
 2016-10       10383   5315    4266   4057       15408          2002     9011   
 2016-11       10383   5315    4265   4057       15406          2002     9011   
 2016-12       10382   5315    4265   4057       15404          2002     9011   
 2017-01       10382   5655    4265   4057       15393          2002     9011   
 2017-02       10382   5655    4264   4057       15392          2002     9011   
 ...             ...    ...     ...    ...         ...           ...      ...   
 2022-03       25223   9711   10788   7988       47292          6924    17444   
 2022-04       25456   9781   10932   8095       48020          7004    17606   
 2022-05       25619   9833   10977   8122       48155          7026    17703   
 2022-06       25836   9884   11069   8161       48616          7090    17783   
 2022-07       26174  10019   11163   8250       49096          7169    17951   
 
          Lamphun   AHB_1 

In [ ]:
# split a univariate sequence into samples
def uni_split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


    # split a multivariate sequence into samples
def multi_split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


def get_scaler(scaler):
    scalers = {
        "minmax": MinMaxScaler,
        "standard": StandardScaler,
        "maxabs": MaxAbsScaler,
        "robust": RobustScaler,
    }
    return scalers.get(scaler.lower())()


# Add attention layer to the deep learning network
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)
 
    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)
 
    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

In [ ]:
%matplotlib inline
# define scope of configs
AHB_list = [7, 8, 9 ,10, 11, 12, 13]
RNN_hidden_units = [8, 16, 32, 64, 128]
n_layer = [1, 2, 3]
n_batch = [2, 4, 8, 16, 32, 64, 128]
n_steps = [5]
patience_list = [3, 5, 7, 10]
mul_uni = ['uni','mul']
dropouts = [0.2, 0.3, 0.5]


timezone_offset = 7.0  # Bangkok Time (UTC+07:00)
tzinfo = timezone(timedelta(hours=timezone_offset))
count = 1
performance_json = []
for g in AHB_list:
    for h in patience_list:
        for a in n_steps:
            for b in dropouts:
                for c in RNN_hidden_units:
                    for d in n_layer:
                        for e in n_batch:
                            for f in mul_uni:

                                print('PROGRESS: [ ' + str(count) + ' / ' + str(len(dropouts)*len(AHB_list)*len(n_steps)*len(RNN_hidden_units)*len(n_layer)*len(n_batch)*len(patience_list)*len(mul_uni)) + ' ]')
                                    
                                if f == 'uni':
                                    if g == 13:
                                        raw_seq = list(df_list[g]['Country_Level'])
                                        ahb = 'Country_Level'
                                    else:    
                                        raw_seq = list(df_list[g]['AHB_'+str(g+1)])
                                        ahb = 'AHB_'+str(g+1)
                                    n_features = 1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq).reshape(len(raw_seq),-1))
                                    X, y = uni_split_sequence(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))

                                if f == 'mul':
                                    if g == 13:
                                        raw_seq = list(df_list[13].drop(df_list[13].columns[:77], axis=1).values)
                                        ahb = 'Country_Level'
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[13]['Country_Level']).reshape(len(df_list[13]['Country_Level']),-1))
                                    else:    
                                        raw_seq = list(df_list[g].drop(['Country_Level'], axis=1).values)
                                        ahb = 'AHB_'+str(g+1)
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[g]['AHB_'+str(g+1)]).reshape(len(df_list[13]['AHB_'+str(g+1)]),-1))
                                    n_features = len(raw_seq[0])-1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq))
                                    X, y = multi_split_sequences(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))


                                callback = EarlyStopping(monitor='loss', patience=h)

                                x=Input(shape=(a,n_features))
                                if d == 1:
                                    RNN_layer = SimpleRNN(c, return_sequences=True, activation='tanh', dropout=b)(x)
                                    attention_layer = attention()(RNN_layer)
                                if d == 2:
                                    RNN_layer = SimpleRNN(c, return_sequences=True, activation='tanh', dropout=b)(x)
                                    RNN_layer2 = SimpleRNN(c, return_sequences=True, activation='tanh', dropout=b)(RNN_layer)
                                    attention_layer = attention()(RNN_layer2)
                                if d ==3:
                                    RNN_layer = SimpleRNN(c, return_sequences=True, activation='tanh', dropout=b)(x)
                                    RNN_layer2 = SimpleRNN(c, return_sequences=True, activation='tanh', dropout=b)(RNN_layer)
                                    RNN_layer3 = SimpleRNN(c, return_sequences=True, activation='tanh', dropout=b)(RNN_layer2)
                                    attention_layer = attention()(RNN_layer3)
                                outputs=Dense(1, trainable=True, activation='tanh')(attention_layer)
                                model=Model(x,outputs)
                                model.compile(loss='mse', optimizer='adam')  
                                history = model.fit(X_train, y_train, epochs=2000, batch_size=e, verbose=0, callbacks=[callback])
                                last_epoch = len(history.history['loss']) 

                                yhat = model.predict(X_test, verbose=0)
                                mae = mean_absolute_error(y_test, yhat)
                                mse = mean_squared_error(y_test, yhat)
                                rmse = mean_squared_error(y_test, yhat, squared = False)
                                mape = mean_absolute_percentage_error(y_test, yhat)
                                    
                                if f == 'uni':
                                    yhat_inverse = scaler.inverse_transform(yhat)
                                    y_test_inverse = scaler.inverse_transform(y_test)
                                if f == 'mul':
                                    yhat_inverse = scaler_y.inverse_transform(yhat)
                                    y_test_inverse = scaler_y.inverse_transform(y_test.reshape(len(y_test),-1))


                                mae_inverse = mean_absolute_error(y_test_inverse, yhat_inverse)
                                mse_inverse = mean_squared_error(y_test_inverse, yhat_inverse)
                                rmse_inverse = mean_squared_error(y_test_inverse, yhat_inverse, squared = False)
                                mape_inverse = mean_absolute_percentage_error(y_test_inverse, yhat_inverse)

                                performance_json.append({'model': 'RNN_with_Attention', 'AHB_number': ahb, 'Multi_Uni': f, \
                                                        'n_layer': d, 'RNN_hidden_units': c, 'dropouts': b,'n_batch': e,\
                                                        'n_steps': a, 'last_epoch': last_epoch, 'patience_list': h, 'MAE_normalize': float(mae), \
                                                        'MSE_normalize': float(mse), 'RMSE_normalize': float(rmse), 'MAPE_normalize': float(mape), \
                                                        'MAE': float(mae_inverse), 'MSE': float(mse_inverse), 'RMSE': float(rmse_inverse), 'MAPE': float(mape_inverse)})
                                    

                                json_formatted_str = json.dumps(performance_json[-1], indent=2)
                                print(json_formatted_str)
                                print(json_formatted_str)
                                print('Percent CPU Usage: ' + str(psutil.cpu_percent()))
                                print('Percent Ram Usage: ' + str(psutil.virtual_memory()[2]))
                                print(datetime.now(tzinfo))

                                count = count + 1

PROGRESS: [ 1 / 17640 ]
{
  "model": "RNN_with_Attention",
  "AHB_number": "AHB_8",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "RNN_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 2,
  "n_steps": 5,
  "last_epoch": 23,
  "patience_list": 3,
  "MAE_normalize": 0.1993092692357375,
  "MSE_normalize": 0.04021641370250683,
  "RMSE_normalize": 0.2005403044340634,
  "MAPE_normalize": 0.20825579741764627,
  "MAE": 7648.69308035714,
  "MSE": 59227424.90981616,
  "RMSE": 7695.935609775862,
  "MAPE": 0.07442701436807585
}
{
  "model": "RNN_with_Attention",
  "AHB_number": "AHB_8",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "RNN_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 2,
  "n_steps": 5,
  "last_epoch": 23,
  "patience_list": 3,
  "MAE_normalize": 0.1993092692357375,
  "MSE_normalize": 0.04021641370250683,
  "RMSE_normalize": 0.2005403044340634,
  "MAPE_normalize": 0.20825579741764627,
  "MAE": 7648.69308035714,
  "MSE": 59227424.90981616,
  "RMSE": 7695.935609775862,
  "MAPE": 0.074427014368

{
  "model": "RNN_with_Attention",
  "AHB_number": "AHB_8",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "RNN_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 8,
  "n_steps": 5,
  "last_epoch": 24,
  "patience_list": 3,
  "MAE_normalize": 0.29915021679563913,
  "MSE_normalize": 0.0901131155722582,
  "RMSE_normalize": 0.3001884667542346,
  "MAPE_normalize": 0.31283173574912143,
  "MAE": 11480.187499999998,
  "MSE": 132711123.70352603,
  "RMSE": 11520.031410700494,
  "MAPE": 0.11174248817020747
}
{
  "model": "RNN_with_Attention",
  "AHB_number": "AHB_8",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "RNN_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 8,
  "n_steps": 5,
  "last_epoch": 24,
  "patience_list": 3,
  "MAE_normalize": 0.29915021679563913,
  "MSE_normalize": 0.0901131155722582,
  "RMSE_normalize": 0.3001884667542346,
  "MAPE_normalize": 0.31283173574912143,
  "MAE": 11480.187499999998,
  "MSE": 132711123.70352603,
  "RMSE": 11520.031410700494,
  "MAPE": 0.11174248817020747
}
Percent 

Streaming output truncated to the last 5000 lines.
  "MAE": 79861.03571428571,
  "MSE": 6559918854.160714,
  "RMSE": 80993.3259852978,
  "MAPE": 0.06513031282630008
}
{
  "model": "RNN_with_Attention",
  "AHB_number": "Country_Level",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "RNN_hidden_units": 32,
  "dropouts": 0.5,
  "n_batch": 128,
  "n_steps": 5,
  "last_epoch": 26,
  "patience_list": 10,
  "MAE_normalize": 0.12420224842685634,
  "MSE_normalize": 0.01586673263751148,
  "RMSE_normalize": 0.1259632193837212,
  "MAPE_normalize": 0.129092033251145,
  "MAE": 79861.03571428571,
  "MSE": 6559918854.160714,
  "RMSE": 80993.3259852978,
  "MAPE": 0.06513031282630008
}
Percent CPU Usage: 53.4
Percent Ram Usage: 12.6
2022-10-08 12:08:20.628961+07:00
PROGRESS: [ 17528 / 17640 ]
{
  "model": "RNN_with_Attention",
  "AHB_number": "Country_Level",
  "Multi_Uni": "mul",
  "n_layer": 1,
  "RNN_hidden_units": 32,
  "dropouts": 0.5,
  "n_batch": 128,
  "n_steps": 5,
  "last_epoch": 24,
  "patience_list

In [ ]:
json.dump(performance_json, open("all_performance_RNN_with_Attention_5nsteps_AHB8-9-10-11-12-13-C.json","w"))
df_json = pd.read_json('/content/all_performance_RNN_with_Attention_5nsteps_AHB8-9-10-11-12-13-C.json')
df_json.to_csv('/content/all_performance_RNN_with_Attention_5nsteps_AHB8-9-10-11-12-13-C.csv', index=False)
#! cp /content/all_performance_RNN_with_Attention_5nsteps_AHB8-9-10-11-12-13-C.json /content/drive/MyDrive/Depression
! cp /content/all_performance_RNN_with_Attention_5nsteps_AHB8-9-10-11-12-13-C.csv /content/drive/MyDrive/Depression

In [ ]:
df_json

,model,AHB_number,Multi_Uni,n_layer,RNN_hidden_units,dropouts,n_batch,n_steps,last_epoch,patience_list,MAE_normalize,MSE_normalize,RMSE_normalize,MAPE_normalize,MAE,MSE,RMSE,MAPE
0,RNN_with_Attention,AHB_8,uni,1,8,0.2,2,5,23,3,0.199309,0.040216,0.200540,0.208256,7648.693080,5.922742e+07,7695.935610,0.074427
1,RNN_with_Attention,AHB_8,mul,1,8,0.2,2,5,21,3,0.055408,0.003489,0.059064,0.057462,2126.347098,5.137582e+06,2266.623590,0.020635
2,RNN_with_Attention,AHB_8,uni,1,8,0.2,4,5,15,3,0.202782,0.041572,0.203893,0.211925,7781.964286,6.122409e+07,7824.582267,0.075729
3,RNN_with_Attention,AHB_8,mul,1,8,0.2,4,5,23,3,0.125359,0.016224,0.127373,0.130725,4810.779018,2.389302e+07,4888.048406,0.046779
4,RNN_with_Attention,AHB_8,uni,1,8,0.2,8,5,24,3,0.299150,0.090113,0.300188,0.312832,11480.187500,1.327111e+08,11520.031411,0.111742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17635,RNN_with_Attention,Country_Level,mul,3,128,0.5,32,5,41,10,0.042195,0.002288,0.047836,0.043403,27131.017857,9.460774e+08,30758.370718,0.022012
17636,RNN_with_Attention,Country_Level,uni,3,128,0.5,64,5,59,10,0.247270,0.061349,0.247688,0.257811,158992.321429,2.536423e+10,159261.514032,0.129870
17637,RNN_with_Attention,Country_Level,mul,3,128,0.5,64,5,39,10,0.031390,0.001467,0.038304,0.032139,20183.339286,6.065842e+08,24628.929890,0.016337
17638,RNN_with_Attention,Country_Level,uni,3,128,0.5,128,5,43,10,0.318982,0.101899,0.319217,0.332767,205102.982143,4.212919e+10,205253.968994,0.167582
